<img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px;" />

# 2.3 Data Access and Basic Processing

## Filter Operators

When interacting with large data collections, it is necessary to keep in mind that it's not possible to load everything!

Therefore, we always have to define our requirements in advance and apply them to the data using filter operators.

Let's start again with the same sample data from the Sentinel-2 STAC Collection:

In [3]:
import openeo
from openeo.local import LocalConnection
local_conn = LocalConnection('')

url = "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a"
spatial_extent = {"west": 11.1, "east": 11.5, "south": 46.1, "north": 46.5}

datacube = local_conn.load_stac(url=url,
                    spatial_extent=spatial_extent)
datacube.execute()

C:\Users\mclaus\Documents\GitHub\openeo-python-client\openeo\metadata.py:255: UserWarning: No cube:dimensions metadata
  complain("No cube:dimensions metadata")
Deserialised process graph into nested structure


Walking node root-6396a979-0cbe-4a5b-8f31-a0031983ca63


C:\Users\mclaus\Documents\GitHub\stackstac\stackstac\prepare.py:364: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-65913ea5da27ab466f462a60b386f389' (time: 804,
                                                                band: 32,
                                                                y: 4534, x: 3209)>
dask.array<getitem, shape=(804, 32, 4534, 3209), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/53)
  * time                                     (time) datetime64[ns] 2016-11-05...
    id                                       (time) <U24 'S2A_32TPS_20161105_...
  * band                                     (band) <U12 'aot' ... 'wvp-jp2'
  * x                                        (x) float64 6.611e+05 ... 6.932e+05
  * y                                        (y) float64 5.153e+06 ... 5.107e+06
    s2:degraded_msi_data_percentage          (time) object 0 0 ... 0.0779 0.0533
    ...                                       ...
    title                                    (band) <U31 'Aerosol optical thi...
    gsd                                      (band) object None 10 ... None None
    common_name                              (band) object None 'blue' ... None
    center_wavelength                        (band) object None 0.49 ... None
    full_width_half_max                      (band) object None 0.098 ... None
    epsg                                     int32 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 5090220.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5200020.00|\n| 0.0...
    resolution:  10.0

### Temporal filter

To slice along time the required data collection with openEO, we can use the `filter_temporal` process.

In [4]:
temporal_extent = ['2022-05-10T00:00:00','2022-06-30T00:00:00']
temporal_slice = datacube.filter_temporal(temporal_extent)
temporal_slice.execute()

Deserialised process graph into nested structure


Walking node root-e2802db4-dc0f-49ff-b67e-ec5a3e7c0708
Walking node loadstac1-e2802db4-dc0f-49ff-b67e-ec5a3e7c0708


C:\Users\mclaus\Documents\GitHub\stackstac\stackstac\prepare.py:364: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


2022-06-30T10:28:08.445000000


<xarray.DataArray 'stackstac-d3fe68627611fbee626aebc803eab0f4' (time: 21,
                                                                band: 32,
                                                                y: 4534, x: 3209)>
dask.array<getitem, shape=(21, 32, 4534, 3209), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/53)
  * time                                     (time) datetime64[ns] 2022-05-11...
    id                                       (time) <U24 'S2A_32TPS_20220511_...
  * band                                     (band) <U12 'aot' ... 'wvp-jp2'
  * x                                        (x) float64 6.611e+05 ... 6.932e+05
  * y                                        (y) float64 5.153e+06 ... 5.107e+06
    s2:degraded_msi_data_percentage          (time) object 0.0245 ... 0.0034
    ...                                       ...
    title                                    (band) <U31 'Aerosol optical thi...
    gsd                                      (band) object None 10 ... None None
    common_name                              (band) object None 'blue' ... None
    center_wavelength                        (band) object None 0.49 ... None
    full_width_half_max                      (band) object None 0.098 ... None
    epsg                                     int32 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 5090220.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5200020.00|\n| 0.0...
    resolution:  10.0

After running the previous cell, it is visible that the result has less elements (or labels) in the temporal dimension `time`.

Additionally, the size of the selected data reduced a lot.

### Spatial filter

In [5]:
spatial_extent = {"west": 11.259613, "east": 11.406212, "south": 46.461019, "north": 46.522237}

spatial_slice = datacube.filter_bbox(spatial_extent)
spatial_slice.execute()

Deserialised process graph into nested structure


Walking node root-f525cc3f-4695-496c-8700-fe42616be45b
Walking node loadstac1-f525cc3f-4695-496c-8700-fe42616be45b


C:\Users\mclaus\Documents\GitHub\stackstac\stackstac\prepare.py:364: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-d21d50579493d2f3a0e0b07304bead3b' (time: 804,
                                                                band: 32,
                                                                y: 489, x: 1145)>
dask.array<getitem, shape=(804, 32, 489, 1145), dtype=float64, chunksize=(1, 1, 382, 860), chunktype=numpy.ndarray>
Coordinates: (12/53)
  * time                                     (time) datetime64[ns] 2016-11-05...
    id                                       (time) <U24 'S2A_32TPS_20161105_...
  * band                                     (band) <U12 'aot' ... 'wvp-jp2'
  * x                                        (x) float64 6.733e+05 ... 6.848e+05
  * y                                        (y) float64 5.153e+06 ... 5.148e+06
    s2:degraded_msi_data_percentage          (time) object 0 0 ... 0.0779 0.0533
    ...                                       ...
    title                                    (band) <U31 'Aerosol optical thi...
    gsd                                      (band) object None 10 ... None None
    common_name                              (band) object None 'blue' ... None
    center_wavelength                        (band) object None 0.49 ... None
    full_width_half_max                      (band) object None 0.098 ... None
    epsg                                     int32 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 5090220.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5200020.00|\n| 0.0...
    resolution:  10.0

### Bands filter

In [6]:
bands = ["B04","B08"]
# TODO: fix datacube metadata from stac
bands_slice = datacube.filter_bands(bands)
bands_slice.execute()

ValueError: Invalid band name/index 'B04'. Valid names: ['unknown']